In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
import requests

In [30]:
original_df = pd.read_csv("../data/filter_time_data.txt", sep='\t\t', header=None)

/home/wujiaocan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
original_df.columns = [ '电影ID','时间', '当前热度值', '新增热度值','当前点击量','新增点击量','当前评论数','新增评论数','当前点赞数','新增点赞数',
                        '当前踩数','新增踩数','当前评分数','推广位','影片推广位等级','当前小时弹幕数','当前小时新增弹幕数','当前小时评分人数',
                        '当前小时新增评分人数','当前小时五星评分人数','当前小时四星评分人数','当前小时三星评分人数','当前小时二星评分人数',
                        '当前小时一星评分人数','当日爱奇艺播放指数','影片上线时间','导演','编剧','制片','美术','主演']

In [25]:
# original_df.head(10)

In [26]:
Y = original_df['新增点击量'].values.tolist()

In [31]:
df = pd.read_csv("../first_version/cleaned_features/original_df_2.csv", index_col=0)

In [32]:
df.head()

,当前热度值,新增热度值,当前评论数,新增评论数,当前点赞数,新增点赞数,当前踩数,新增踩数,当前评分数,当前小时评分人数,...,hour15,hour16,hour17,hour18,hour19,hour20,hour21,hour22,hour23,heat_value_window_size_6
0,0.446225,-1,412,0,978,0,623,0,690,30091,...,0,0,0,1,0,0,0,0,0,0.0
1,0.445907,-2,412,0,978,0,623,0,690,30091,...,0,0,0,0,1,0,0,0,0,0.0
2,0.445749,-1,412,0,978,0,623,0,690,30091,...,0,0,0,0,0,1,0,0,0,0.0
3,0.445590,-1,412,0,978,0,623,0,690,30091,...,0,0,0,0,0,0,1,0,0,0.0
4,0.444956,-4,412,0,978,0,623,0,690,30091,...,0,0,0,0,0,0,0,1,0,0.0


In [13]:
del df['heat_value_window_size_6']

In [33]:
X = df.values.tolist()

In [35]:
len(X[0])

72

In [27]:
import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [28]:
## 去掉'heat_value_window_size_6'，用默认参数跑

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

# 对测试集进行预测
ans = model.predict(X_test)

print("MSE:",metrics.mean_squared_error(y_test, ans))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, ans))) 

MSE: 3405274.3308854103
RMSE: 1845.3385409960445


In [29]:
## 去掉'heat_value_window_size_6'，用最优参数跑

model = xgb.XGBRegressor(max_depth=7, n_estimators=500, gamma=0.027698143119166907, colsample_bytree=0.8957822073838446)
model.fit(X_train, y_train)

# 对测试集进行预测
ans = model.predict(X_test)

print("MSE:",metrics.mean_squared_error(y_test, ans))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, ans))) 

MSE: 2025883.5443395092
RMSE: 1423.3353590561535


In [36]:
## 保留'heat_value_window_size_6'，用最优参数跑

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

model = xgb.XGBRegressor(max_depth=7, n_estimators=500, gamma=0.027698143119166907, colsample_bytree=0.8957822073838446)
model.fit(X_train, y_train)

# 对测试集进行预测
ans = model.predict(X_test)

print("MSE:",metrics.mean_squared_error(y_test, ans))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, ans))) 

MSE: 2270612.4824847123
RMSE: 1506.855163074644
